In [ ]:
#new dataloader

In [ ]:
#load data like the other embeddings

In [ ]:
import torch
import torchaudio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt

from google.cloud import storage, bigquery

In [ ]:
from speech_utils import *

In [ ]:
project_name = 'ml-mps-aif-afdgpet01-p-6827'
study = 'speech_poc_freeze_1'
bucket_name = 'ml-e107-phi-shared-aif-us-p'
gcs_prefix = f'speech_ai/speech_lake/{study}'

storage_client = storage.Client()
bq_client = bigquery.Client()
bucket = storage_client.bucket(bucket_name)

In [ ]:
target_features = [
    'stridor',
    'breathy',
    'rapid rate',
    'reduced OA loudness',
    'increased OA loudness',
    'high pitch'
]

target_diags = [
    'Ataxic dysarthria',
    'Hypokinetic dysarthria',
    'Hyperkinetic Dysarthria',
    'Spastic dysarthria'
]

In [ ]:
query = f"""
SELECT *
FROM `ml-mps-aif-afdgpet01-p-6827.phi_main_us_p.speech_lake`
WHERE task = 'Alternating Motion Rate'
LIMIT 10
"""
query_df = bq_client.query(query).to_dataframe().set_index('uid')

In [ ]:
binarize = lambda v: 1 if v > 1.0 else 0
def rank(v):
    if v < 0 or v > 5:
        return np.nan
    else:
        return v - 1

In [ ]:
features_df = query_df.apply(lambda r: vectorize_scores(r['features'], targets = target_features, map_fn = rank), axis = 1)
diag_df = query_df.apply(lambda r: vectorize_scores(r['diag'], targets = target_diags, map_fn = binarize), axis = 1)


In [ ]:
channel_sum = lambda w: torch.sum(w, axis = 0).unsqueeze(0)

In [ ]:
waveform_loader = UidToWaveform(bucket = bucket, gcs_prefix = gcs_prefix)
truncate_tfm = Truncate(length = 160000, offset = 100)
mono_tfm = ToMonophonic(reduce_fn = channel_sum)
tensor_tfm = ToTensor()
downsample_tfm = Resample(resample_rate = 16000)
mfcc_tfm = MFCC(n_mfcc=80, n_fft=400, n_mels=80)
mel_transforms = MelSpectrogram(n_fft=400, n_mels=80)
transform = transforms.Compose([waveform_loader, downsample_tfm, truncate_tfm, mono_tfm, mel_transforms, mfcc_tfm, tensor_tfm])

In [ ]:
waveform_loader = UidToWaveform(bucket = bucket, gcs_prefix = gcs_prefix)
mono_tfm = ToMonophonic(reduce_fn = channel_sum)
downsample_tfm = Resample(16000)
trim_tfm = TrimSilence()
tensor_tfm = ToTensor()
truncate_tfm = Truncate(length = 500000)
transform = transforms.Compose([waveform_loader, mono_tfm, downsample_tfm, trim_tfm, truncate_tfm, tensor_tfm])

In [ ]:
waveform_dataset = WaveformDataset(annotations_df = features_df, target_labels = target_features, transform = transform)

In [ ]:
dataloader = DataLoader(waveform_dataset, batch_size=len(features_df), shuffle=False, num_workers=0)